___

# GridSearch: PyGSP for Mouse Phenotype prediction

In [1]:
from sklearn.metrics import mean_squared_error,r2_score,mean_absolute_error,accuracy_score
from pygsp import graphs, filters, plotting, learning
from scipy.spatial.distance import pdist, squareform
import matplotlib.pyplot as plt
from tabulate import tabulate
from scipy.stats import norm
import networkx as nx
import pandas as pd
import numpy as np
import scipy
import os
import re
import pickle

## Import Data

<div class="alert alert-block alert-info">
    
* genotype_df: contains the genotype of each strain for each allele
* phenotype_df : contains the phenotypes of each strain, lots of Nan

NB: on this branch of the project, we don't use the expression of the genes, and try instead to predict straight the phenotype from the genotype: therefore, we don't import any file from expression data.
</div>

In [2]:
genotype_df = pd.read_csv("data/genotype_BXD.txt", sep='\t', index_col='SNP')

# Remove F0 and F1 generation (parental)
genotype_df.drop(['B6D2F1', 'D2B6F1', 'C57BL.6J', 'DBA.2J'], axis=1, inplace=True)

genotype_df = genotype_df.transpose()
genotype_df.index.name = 'BXD_strain'

print("There is NaN values: %s" % genotype_df.isna().any().any())

phenotype_df = pd.read_csv("data/Phenotype.txt", sep='\t', index_col='PhenoID')

# Remove F0 and F1 generation (parental)
phenotype_df.drop(['B6D2F1', 'D2B6F1', 'C57BL.6J', 'DBA.2J'], axis=1, inplace=True)

nan_count = phenotype_df.isna().sum().sum()
entries_count = phenotype_df.shape[0] * phenotype_df.shape[1]

print("Number of Nan values: %s" % nan_count)
print("Percentage of nan in the phenotype file: {:0.2f}%".format(nan_count / entries_count * 100))

There is NaN values: False
Number of Nan values: 291981
Percentage of nan in the phenotype file: 61.66%


### Phenotype_id_aligner

In [3]:
# Phenotype id aligne is an in-depth description of each phenotype
phenotype_id_aligner = pd.read_csv('data/phenotypes_id_aligner.txt', sep='\t', encoding='latin1')
phenotype_id_aligner.rename({'Quantitive.trait': 'quantitative_trait'}, axis=1, inplace=True)
quant_pheno_best = phenotype_id_aligner[(phenotype_id_aligner.Strains > 70) & (phenotype_id_aligner.quantitative_trait == 'No')]
quant_pheno_best = quant_pheno_best.sort_values(by='Strains', ascending=False)

#### Weighted edges

In [4]:
def epsilon_similarity_graph(X: np.ndarray, metric='euclidean', sigma=1, epsilon=0):
    """ X (n x d): coordinates of the n data points in R^d.
        sigma (float): width of the kernel
        epsilon (float): threshold
        Return:
        adjacency (n x n ndarray): adjacency matrix of the graph.
    """
    Dists = squareform(pdist(X,metric = "euclidean"))
    Dists = np.exp(-Dists**2/(2 * sigma**2))
    Dists[Dists <= epsilon] = 0
    np.fill_diagonal(Dists,0)
    return Dists

# Exploitation

TODO:
* Throw a big gridsearch on many parameters
* adapt the grid_search_graphs function s.t. it can compute the adjacency matrix with the gene expression data.

<div class="alert alert-block alert-info">
At that point, all we need is a pygsp graph describing the different strains: the way it is linked is therefore critical for the success of the algorithm.
</div>

In [5]:
#small parameters for tests
METRICS = ['cosine']
SIGMAS = [60,70]
EPSILONS = [0.28]
TAUS = [0.1]

In [6]:
#real parameters
METRICS = ['cosine','jaccard','hamming','euclidian']
SIGMAS = np.arange(60,76,15)
EPSILONS = np.arange(0.2,0.4,0.01)
TAUS = np.logspace(0,10,10)

In [7]:
def get_min_max(params_scores):
    ind_max = np.nanargmax([i[0] for i in params_scores])
    ind_min = np.nanargmin([i[0] for i in params_scores])
    maxo = params_scores[ind_max][0]
    mino = params_scores[ind_min][0]
    return mino,maxo

In [8]:
phenotype_df['nan_count'] = phenotype_df.isnull().sum(axis = 1)
top_variables = phenotype_df.join(phenotype_id_aligner.set_index('PhenoID')[['quantitative_trait']],how = 'inner')
top_continuous = top_variables[top_variables.quantitative_trait == 'Yes'].sort_values('nan_count').head(30).copy()
top_discrete = top_variables[top_variables.quantitative_trait == 'No'].sort_values('nan_count').head(30).copy()
print('The best continuous variables are : {}'.format(' '.join(top_continuous.index.values)))
print('The best discrete variables are : {}'.format(' '.join(top_discrete.index.values)))

The best continuous variables are : X3820 X2397 X1002 X328 X218 X416 X699 X1302 X2147 X2164 X885 X1206 X2334 X2230 X1646 X1977 X424 X2041 X4482 X332 X228 X3903 X4556 X4256 X4849 X5029 X2490 X457 X2645 X2689
The best discrete variables are : X546 X152 X111 X1012 X4473 X64 X62 X63 X76 X65 X545 X66 X69 X88 X70 X67 X553 X461 X646 X343 X4472 X215 X414 X61 X1011 X110 X57


In [9]:
def grid_search_graphs(CV_meth,metrics,sigmas,epsilons,taus,gene):
    """
    CV_meth: a method performing cross validation (on discrete or continuous variable)
    metrics: list of metrics usable in a pdist matrix
    sigmas,epsilons: parameter for the adjacency matrix we do the grid search on
    """
    params_scores = []
    tot_comb = len(metrics) * len(sigmas) * len(epsilons) * len(taus)
    for i,met in enumerate(metrics):
        for j,sig in enumerate(sigmas):
            for k,eps in enumerate(epsilons):
                for l,tau in enumerate(taus):
                    ind = l + len(taus) * (k + len(epsilons) * (j + len(sigmas) * (i)))
                    if (ind % 10 == 0):
                        print(" %i / %i combinations" % (ind,tot_comb))
                    try:
                        weighted_adjacency = epsilon_similarity_graph(genotype_df.values, metric=met, sigma=sig, epsilon=eps)
                        pygsp_weighted_graph = graphs.Graph(weighted_adjacency,lap_type='normalized') 
                        res = CV_meth(pygsp_weighted_graph,phenotype_df,genotype_df,tau,gene)
                        params_scores.append([res,(met,sig,eps,tau)])
                    except nx.NetworkXError:
                        print('non connected graph with %.2f sigma and %.2f epsilon' % (sig,eps))
                        continue
    mino,maxo = get_min_max(params_scores)
    print('#' * 32)
    print('Biggest result for gene %s : %.2f' % (gene,maxo))
    print('Smallest result for gene %s : %.2f' % (gene,mino))
    print('#' * 32)
    return params_scores
    

### Labels (hair coat color)

In [10]:
def CV_hair_label(pygsp_weighted_graph,phenotype_df,genotype_df,tau,gene = 'X62',NUM_REPET = 50,plot = False):
    y_true = phenotype_df.loc[gene][genotype_df.index].fillna(0)
    acc = 0
    for i in range(NUM_REPET):
        rs = np.random.RandomState()
        mask = rs.uniform(0,1,pygsp_weighted_graph.N) > 0.2
        labels_true = y_true[~mask]
        y_sparse = y_true.copy()
        y_sparse[~mask] = np.nan

        recovery = learning.classification_tikhonov(pygsp_weighted_graph,y_sparse,mask,tau=tau)
        #we get rid of the 0 value: we don't accept no label

        #recovery[:,0] = -1
        prediction = np.argmax(recovery, axis=1)

        labels_pred = prediction[~mask]
        acc += accuracy_score(labels_true,labels_pred)
    acc_final = acc/NUM_REPET
    if plot:
        print('Accuracy:',acc_final)
        fig, ax = plt.subplots(1, 3, sharey=True, figsize=(20, 6))
        #ground truth
        pygsp_weighted_graph.plot_signal(y_true, ax=ax[0], title='Ground truth')
        #measurements
        pygsp_weighted_graph.plot_signal(y_sparse, ax=ax[1], title='Measurements')
        #recover
        pygsp_weighted_graph.plot_signal(prediction, ax=ax[2], title='Recovered class')
    return acc_final

In [11]:
list_genes = top_discrete.index.values
dic_disc = {}
for gene in list_genes:
    result = np.nan
    try:
        result = grid_search_graphs(CV_hair_label,METRICS,SIGMAS,EPSILONS,TAUS,gene)
        dic_disc[gene] = result
        with open('results/dic_disc.pickle','wb') as f:
            pickle.dump(dic_disc,f)
    except:
        continue

 0 / 1600 combinations
 10 / 1600 combinations
 20 / 1600 combinations
 30 / 1600 combinations
 40 / 1600 combinations
 50 / 1600 combinations
 60 / 1600 combinations
 70 / 1600 combinations
 80 / 1600 combinations
 90 / 1600 combinations
 100 / 1600 combinations
 110 / 1600 combinations
 120 / 1600 combinations
 130 / 1600 combinations
 140 / 1600 combinations
 150 / 1600 combinations
 160 / 1600 combinations
 170 / 1600 combinations
 180 / 1600 combinations
 190 / 1600 combinations
 200 / 1600 combinations
 210 / 1600 combinations
 220 / 1600 combinations
 230 / 1600 combinations
 240 / 1600 combinations
 250 / 1600 combinations
 260 / 1600 combinations
 270 / 1600 combinations
 280 / 1600 combinations
 290 / 1600 combinations
 300 / 1600 combinations
 310 / 1600 combinations
 320 / 1600 combinations
 330 / 1600 combinations
 340 / 1600 combinations
 350 / 1600 combinations
 360 / 1600 combinations
 370 / 1600 combinations
 380 / 1600 combinations
 390 / 1600 combinations
 400 / 1600

### Continuous variable

As we can see, there are more than 10 continuous phenotype who have interesting charateristics as continuous features.

In [12]:
def CV_cont_phen(pygsp_weighted_graph,phenotype_df,genotype_df,tau,gene,NUM_REPET = 50,metric_meth = r2_score,plot = False):
    ycont_true = phenotype_df.loc[gene][genotype_df.index]
    ycont_true.fillna(ycont_true.mean(),inplace = True)
    #mae or mse
    mxe = 0
    for i in range(NUM_REPET):
        mask = np.random.RandomState().uniform(0,1,pygsp_weighted_graph.N) > 0.2
        vals_true = ycont_true[~mask]
        ycont_sparse = ycont_true.copy()
        ycont_sparse[~mask] = np.nan

        recovery = learning.regression_tikhonov(pygsp_weighted_graph,ycont_sparse,mask,tau=tau)
        if any(np.isnan(recovery)):
            return np.nan
        vals_pred = recovery[~mask]
        mxe += metric_meth(vals_true,vals_pred)

    mxe_avg = mxe / NUM_REPET
    if plot:
        print(str(metric_meth),mxe_avg)
        fig, ax = plt.subplots(1, 3, sharey=True, figsize=(20, 6))
        limits = [ycont_true.min(),ycont_true.max()]

        #ground truth
        pygsp_weighted_graph.plot(ycont_true, ax=ax[0],limits = limits, title='Ground truth')
        #measurements
        pygsp_weighted_graph.plot_signal(ycont_sparse, ax=ax[1],limits = limits, title='Measurements')
        #recover
        pygsp_weighted_graph.plot_signal(recovery, ax=ax[2],limits = limits, title='Recovered class')
    return mxe_avg

In [13]:
phenotype_df['nan_count'] = phenotype_df.isnull().sum(axis = 1)
top_continuous = phenotype_df.join(phenotype_id_aligner.set_index('PhenoID')[['quantitative_trait']],how = 'inner')
top_continuous = top_continuous[top_continuous.quantitative_trait == 'Yes'].sort_values('nan_count').head(30)
print('The best continuous variable are : {}'.format(' '.join(top_continuous.index.values)))

The best continuous variable are : X3820 X2397 X1002 X328 X218 X416 X699 X1302 X2147 X2164 X885 X1206 X2334 X2230 X1646 X1977 X424 X2041 X4482 X332 X228 X3903 X4556 X4256 X4849 X5029 X2490 X457 X2645 X2689


In [14]:
list_cont_genes = top_continuous.index.values
dic_cont = {}
for gene in list_cont_genes:
    try:
        results_cont = grid_search_graphs(CV_cont_phen,METRICS,SIGMAS,EPSILONS,TAUS,gene)
        ind_max = np.nanargmax([i[0] for i in results_cont])
        r2 = results_cont[ind_max]
        dic_cont[gene] = results_cont
        with open('results/dic_cont.pickle','wb') as f:
            pickle.dump(dic_cont,f)
    except:
        continue

 0 / 1600 combinations
 10 / 1600 combinations
 20 / 1600 combinations
 30 / 1600 combinations
 40 / 1600 combinations
 50 / 1600 combinations
 60 / 1600 combinations
 70 / 1600 combinations
 80 / 1600 combinations
 90 / 1600 combinations
 100 / 1600 combinations
 110 / 1600 combinations
 120 / 1600 combinations
 130 / 1600 combinations
 140 / 1600 combinations
 150 / 1600 combinations
 160 / 1600 combinations
 170 / 1600 combinations
 180 / 1600 combinations
 190 / 1600 combinations
 200 / 1600 combinations
 210 / 1600 combinations
 220 / 1600 combinations
 230 / 1600 combinations
 240 / 1600 combinations
 250 / 1600 combinations
 260 / 1600 combinations
 270 / 1600 combinations
 280 / 1600 combinations
 290 / 1600 combinations
 300 / 1600 combinations
 310 / 1600 combinations
 320 / 1600 combinations
 330 / 1600 combinations
 340 / 1600 combinations
 350 / 1600 combinations
 360 / 1600 combinations
 370 / 1600 combinations
 380 / 1600 combinations
 390 / 1600 combinations
 400 / 1600

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [20]:
list_cont_genes

array(['X3820', 'X2397', 'X1002', 'X328', 'X218', 'X416', 'X699', 'X1302',
       'X2147', 'X2164', 'X885', 'X1206', 'X2334', 'X2230', 'X1646',
       'X1977', 'X424', 'X2041', 'X4482', 'X332', 'X228', 'X3903',
       'X4556', 'X4256', 'X4849', 'X5029', 'X2490', 'X457', 'X2645',
       'X2689'], dtype=object)

In [16]:
get_min_max(dic_cont['X218'])

(-539.9268262347956, -0.1772958989724001)

In [23]:
for i in dic_disc.keys():
    print(i,get_min_max(dic_disc[i]))

X546 (0.06716521045174292, 0.7524638995485006)
X152 (0.15557868197446198, 0.655744472642613)
X111 (0.0679616678295244, 0.6535484335299518)
X1012 (0.06652731842124761, 0.7758423098999085)
X4473 (0.01183170424346895, 0.12305522562245538)
X64 (0.0873975795904588, 0.4944742765759389)
X62 (0.08174949191339013, 0.49153061426391625)
X63 (0.07181065373133617, 0.5189940826997295)
X76 (0.32781348619332706, 0.7096777387857952)
X553 (0.2351610675659673, 0.447551846995877)
X461 (0.22937760422374764, 0.46026208721731005)
X646 (0.24179709344662897, 0.4643544190863952)
X343 (0.22355982400812124, 0.47768812281330786)
X61 (0.6235783666006732, 0.7847612191924078)
X57 (1.0, 1.0)


In [25]:
[i for i in dic_disc['X61'] if i[0]> 0.78]

[[0.7847612191924078, ('cosine', 75, 0.3200000000000001, 59948425.03189421)]]

In [30]:
weighted_adjacency = epsilon_similarity_graph(genotype_df.values, metric='cosine', sigma=75, epsilon=0.32)
pygsp_weighted_graph = graphs.Graph(weighted_adjacency,lap_type='normalized')
CV_hair_label(pygsp_weighted_graph,phenotype_df,genotype_df,59948425,'X61')

0.7469188898553594